# System Workflow

![Screenshot](./Snapshots/flow_diagram.png)

## Step-1 :
## Load the Youtube Transcripts based on TimeStamp Chunks

In [1]:
from langchain_community.document_loaders import YoutubeLoader # Load the Youtube Transcript
from langchain_community.document_loaders.youtube import TranscriptFormat # To Get transcripts as timestamped chunks
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
try:
    loader = YoutubeLoader.from_youtube_url(
        "https://www.youtube.com/watch?v=pJdMxwXBsk0&list=PLKnIA16_RmvaTbihpo4MtzVm4XOQa0ER0&index=15",
         language=["hi"],
         translation="en",
        transcript_format=TranscriptFormat.CHUNKS,
        chunk_size_seconds=60,
    )
    docs = loader.load()

    if docs:
        print(f"Successfully loaded {len(docs)} transcript chunks")
    else:
        print("No transcript data was loaded (empty result)")

except Exception as e:
    print(f"Error loading YouTube transcript: {str(e)}")
    docs = None  

if docs:
    pass
else:
    print("Failed to load transcript, cannot proceed")

Successfully loaded 52 transcript chunks


In [3]:
len(docs)

52

In [4]:
docs[0]

Document(metadata={'source': 'https://www.youtube.com/watch?v=pJdMxwXBsk0&t=0s', 'start_seconds': 0, 'start_timestamp': '00:00:00'}, page_content="Hi guys, my name is Nitesh and welcome to my YouTube channel.  In this video also we will continue our lang chain playlist. And the topic of today's video is retrievers which is a very important topic.  If you talk about rag.  If you want to build a RAG based application then retriever is a very important component.  In fact, in the future when you make some advanced rag systems, you will work with different types of retrievers there, so in that sense this particular video is very important. And I would like you to watch this video end to end.  So in today's video I will not only explain to you what are retrievers?  What do they need?  But at the same time I will also tell you about different types of retrievers and will show you the code.  Ok?  So ya let's start the video.  So guys, before we start the video, I would like to give you a quic

In [5]:
docs[2]

Document(metadata={'source': 'https://www.youtube.com/watch?v=pJdMxwXBsk0&t=120s', 'start_seconds': 120, 'start_timestamp': '00:02:00'}, page_content="how we are moving forward with this playlist. Now let's focus on today's video, which is on retrievers. Retrievers are very important in langche.  So we will cover this in great detail in today's video.  First of all, we will start with this discussion that what are retrievers?  So in very simple words, if you read this first line, it is written here that a retriever is a component in the language that fetches relevant documents from a data source in response to a user's query.  Ok?  If you focus on this diagram, you will understand things better visually. So what happens is that you have a data source where all your data is stored.  Ok ?  All the data related to anything is stupid.  Now this data source can be anything.  It could be a vector store and it could be some API or something.")

In [6]:
index= 0
print(docs[index].metadata)
print(docs[index].page_content)

{'source': 'https://www.youtube.com/watch?v=pJdMxwXBsk0&t=0s', 'start_seconds': 0, 'start_timestamp': '00:00:00'}
Hi guys, my name is Nitesh and welcome to my YouTube channel.  In this video also we will continue our lang chain playlist. And the topic of today's video is retrievers which is a very important topic.  If you talk about rag.  If you want to build a RAG based application then retriever is a very important component.  In fact, in the future when you make some advanced rag systems, you will work with different types of retrievers there, so in that sense this particular video is very important. And I would like you to watch this video end to end.  So in today's video I will not only explain to you what are retrievers?  What do they need?  But at the same time I will also tell you about different types of retrievers and will show you the code.  Ok?  So ya let's start the video.  So guys, before we start the video, I would like to give you a quick recap of what we have been doin

## Step-2
## Loading the embedding model and the llm

In [7]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

In [8]:
embedding_model = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-mpnet-base-v2")
from langchain_huggingface import ChatHuggingFace,HuggingFaceEndpoint
from transformers import AutoTokenizer
# Initialize a llm model
# repo_id = "mistralai/Mistral-7B-Instruct-v0.3"
# # First load the tokenizer explicitly
# tokenizer = AutoTokenizer.from_pretrained(repo_id)
# llm1 = HuggingFaceEndpoint(
#     repo_id = repo_id,
#     temperature = 0.8,
#     max_new_tokens=500,
# )
# llm = ChatHuggingFace(llm=llm1,tokenizer=tokenizer)

c:\Users\Prabal Kuinkel\Desktop\Youtube-Conversational_AI\youtube_ai\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
# !pip install langchain_groq

In [10]:
from langchain_groq import ChatGroq
llm = ChatGroq(model_name = "Llama-3.3-70b-Versatile",max_tokens= 900)

## Step-3
 
 ## Creating a vectordatabase using the Chroma db


In [11]:
# vector_store = FAISS.from_documents(docs, embedding_model)
from langchain_chroma import Chroma
vectorstore = Chroma.from_documents(docs, embedding_model)

In [12]:
# !pip install langchain-chroma
# !pip install lark

## Step-4 Defining the retriever
## Using the Metadatabased Filtering for retrievers

#### -> this retriever is known as self-query retriever

In [13]:
print(docs[index].metadata)

{'source': 'https://www.youtube.com/watch?v=pJdMxwXBsk0&t=0s', 'start_seconds': 0, 'start_timestamp': '00:00:00'}


In [14]:
from langchain.chains.query_constructor.schema import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
metadata_field_info = [
    AttributeInfo(
        name="source",
        description="The link of the video",
        type="string"
    ),
    AttributeInfo(
        name="start_seconds",
        description="The starting second of the video chunk (in seconds as integer)",
        type="integer"  # Changed from string to integer
    ),
    AttributeInfo(
        name="start_timestamp",
        description="Human-readable timestamp (HH:MM:SS format)",
        type="string"
    )
]

In [15]:
# # First get the base retriever from your vectorstore with increased k
# base_vectorstore_retriever = vectorstore.as_retriever(
#     # search_type = "mmr",
#     search_kwargs={"k": 20,'lambda_mult':0.5}  # Increase this number as needed
# )

In [16]:
document_content_description = "Transcript of a youtube video"
retriever = SelfQueryRetriever.from_llm(
    llm,
    vectorstore,
    document_content_description,
    metadata_field_info,
    # base_retriever = base_vectorstore_retriever,
    verbose=True,
    search_kwargs={"k": 8}  # Increase this number as needed

)

In [17]:
# This example only specifies a filter
# retriever.invoke("Create me a blog post about the video.")
# retriever.invoke("what is meant by multi query retriever ?")

## Step 5 Creating a Prompt Template

In [18]:
from langchain_core.prompts import PromptTemplate

In [19]:
template = PromptTemplate(
    template = """You are an AI assistant , which has access to a youtubes video transcript. Answer the user's query based on the provided transcripts context and do not hallicunate. 
    If the answer to the user'query is not mentioned in the context or incase if you dont know the answer respond with 'Sorry i do not have answer to your question'.
    'context'
    {context}
    'Question'
    {input}""",
    input_variables=['context','input']
)

## Step-6 Creating a RAG Chain

In [20]:
from langchain.chains.combine_documents import create_stuff_documents_chain #"Formats retrieved documents + question into a prompt and passes it to the LLM for answering."
from langchain.chains import create_retrieval_chain # "Combines a retriever (to fetch docs) with the 'create_stuff_document_chain' to automate end-to-end retrieval + answering."

In [21]:
combined_docs_chain = create_stuff_documents_chain(llm=llm,prompt=template)
rag_chain = create_retrieval_chain(retriever,combined_docs_chain)

In [22]:
# result=rag_chain.invoke({
#     'input':"Create me a 8 MCQ questions from the video . Also provide 4 options and also tell which is the correct one ."
# })

In [23]:
# print(result['answer'])

## Step- 7 Creating tools

### Tool A. VectorStore Retriever tool (Convert the rag_chain into a tool)
Redirect to this tool if the user queries is regarding the Video content

### Tool B. DuckDuckSeach Tool
Redirect to this tool if the user query is general

In [24]:
from langchain_core.tools import tool
from langchain_community.tools import DuckDuckGoSearchRun #Search user queries Online

@tool
def retriever_vectorstore_tool(query:str)->str:
    """Use this tool when the user ask about:
    - content of the youtube video
    - Any queries specifically about the youtube video
    Input should be the exact search query.
    The tool will perform a vectorstore search using retriever."""
    # print(query)
    docs = retriever.invoke(query)
    print(f"_______________Retrieved Context from vectorstore_____ {docs}")
    # return rag_chain.invoke({"input":query})
    return docs



search = DuckDuckGoSearchRun()
@tool
def duckducksearch_tool(query: str) -> str:
    """Use this tool when:
    - The question is not about youtube video
    - The question requires current, real-world information
    - The topic is general knowledge
    - The query is about news, weather, or public information
    
    Input should be the exact search query.
    The tool will perform a web search using DuckDuckGo.
    """
    return search.invoke(query)

## Step-8 Creating a react agent using langgraph

In [25]:
# from langgraph.prebuilt import create_react_agent
from langchain.agents import AgentExecutor,create_react_agent


In [26]:
from langchain_core.prompts import PromptTemplate
# prompt = PromptTemplate(
#     template="""
# Answer the following questions as best you can. You have access to the following tools:
# {tools}

# Use the following format:

# Question: the input question you must answer
# Thought: you should always think about what to do
# Action: the action to take, should be one of [{tool_names}]
# Action Input: the input to the action
# Observation: the result of the action
# ... (this Thought/Action/Action Input/Observation can repeat N times)
# Thought: I now know the final answer
# Final Answer: the final answer to the original input question

# Begin!

# Question: {input}
# Thought:{agent_scratchpad}
# """,
# input_variables=['tools','tool_names','input','agent_scratchpad']
# )
# Step 2: Pull the ReAct prompt from LangChain Hub
from langchain import hub
prompt = hub.pull("hwchase17/react")  # pulls the standard ReAct agent prompt


In [27]:
tool = [retriever_vectorstore_tool,duckducksearch_tool]

In [28]:
tool

[StructuredTool(name='retriever_vectorstore_tool', description='Use this tool when the user ask about:\n    - content of the youtube video\n    - Any queries specifically about the youtube video\n    Input should be the exact search query.\n    The tool will perform a vectorstore search using retriever.', args_schema=<class 'langchain_core.utils.pydantic.retriever_vectorstore_tool'>, func=<function retriever_vectorstore_tool at 0x000002611C27F7E0>),
 StructuredTool(name='duckducksearch_tool', description='Use this tool when:\n    - The question is not about youtube video\n    - The question requires current, real-world information\n    - The topic is general knowledge\n    - The query is about news, weather, or public information\n\n    Input should be the exact search query.\n    The tool will perform a web search using DuckDuckGo.', args_schema=<class 'langchain_core.utils.pydantic.duckducksearch_tool'>, func=<function duckducksearch_tool at 0x000002611C27F060>)]

In [29]:

tools = [retriever_vectorstore_tool,duckducksearch_tool]
agent = create_react_agent(
    llm=llm, 
    tools=[retriever_vectorstore_tool,duckducksearch_tool], 
    prompt=prompt)


# Step 4: Wrap it with AgentExecutor
agent_executor = AgentExecutor(
    agent=agent,
    tools=[retriever_vectorstore_tool,duckducksearch_tool],
    verbose=True,
    handle_parsing_errors=True
)

In [36]:
query = "Explain me in detail the summary of the video ?"
messages = agent_executor.invoke({"input":query})



> Entering new AgentExecutor chain...
To provide a detailed summary of the video, I first need to understand what the video is about. 

Action: retriever_vectorstore_tool
Action Input: "video summary"_______________Retrieved Context from vectorstore_____ [Document(id='bdf26d33-c0a0-4a5f-b4a5-2c7aac65d5c1', metadata={'source': 'https://www.youtube.com/watch?v=pJdMxwXBsk0&t=3060s', 'start_seconds': 3060, 'start_timestamp': '00:51:00'}, page_content='application.  Ok?  So with that I will conclude this video.  If you liked the video, please like it.  If you have not subscribed to this channel, please do subscribe.  See you in the next video , bye.'), Document(id='5924e11e-1fc0-41c5-9a3b-52d184620f08', metadata={'source': 'https://www.youtube.com/watch?v=pJdMxwXBsk0&t=120s', 'start_seconds': 120, 'start_timestamp': '00:02:00'}, page_content="how we are moving forward with this playlist. Now let's focus on today's video, which is on retrievers. Retrievers are very important in langche.  S

In [37]:
print(messages['output'])

The video discusses retrievers in the context of language models and vector stores. It covers the basics of retrievers, their importance in langchain, and different types of retrievers such as Maximum Marginal Relevance, Contextual Compression Retriever, and others. The video also provides code examples and explanations of how to use these retrievers. Retriever types can be categorized based on the data source they work with and the search strategy they use. There are many retrievers available in Langchen, including Wikipedia Retriever, Vector Store based retrievers, and others. The video provides a detailed explanation of retrievers and their use cases, and it is an important topic for building RAG-based applications.


In [41]:
retriever_vectorstore_tool.invoke("Explain me in detail the summary of the video ?")

_______________Retrieved Context from vectorstore_____ [Document(id='bdf26d33-c0a0-4a5f-b4a5-2c7aac65d5c1', metadata={'source': 'https://www.youtube.com/watch?v=pJdMxwXBsk0&t=3060s', 'start_seconds': 3060, 'start_timestamp': '00:51:00'}, page_content='application.  Ok?  So with that I will conclude this video.  If you liked the video, please like it.  If you have not subscribed to this channel, please do subscribe.  See you in the next video , bye.'), Document(id='5924e11e-1fc0-41c5-9a3b-52d184620f08', metadata={'source': 'https://www.youtube.com/watch?v=pJdMxwXBsk0&t=120s', 'start_seconds': 120, 'start_timestamp': '00:02:00'}, page_content="how we are moving forward with this playlist. Now let's focus on today's video, which is on retrievers. Retrievers are very important in langche.  So we will cover this in great detail in today's video.  First of all, we will start with this discussion that what are retrievers?  So in very simple words, if you read this first line, it is written he

[Document(id='bdf26d33-c0a0-4a5f-b4a5-2c7aac65d5c1', metadata={'source': 'https://www.youtube.com/watch?v=pJdMxwXBsk0&t=3060s', 'start_seconds': 3060, 'start_timestamp': '00:51:00'}, page_content='application.  Ok?  So with that I will conclude this video.  If you liked the video, please like it.  If you have not subscribed to this channel, please do subscribe.  See you in the next video , bye.'),
 Document(id='5924e11e-1fc0-41c5-9a3b-52d184620f08', metadata={'source': 'https://www.youtube.com/watch?v=pJdMxwXBsk0&t=120s', 'start_seconds': 120, 'start_timestamp': '00:02:00'}, page_content="how we are moving forward with this playlist. Now let's focus on today's video, which is on retrievers. Retrievers are very important in langche.  So we will cover this in great detail in today's video.  First of all, we will start with this discussion that what are retrievers?  So in very simple words, if you read this first line, it is written here that a retriever is a component in the language tha

In [33]:
print(prompt.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}
